# Projekt 1.

## Przygotowanie danych.

In [27]:
import pandas as pd
import numpy as np


dane = pd.read_excel("loty_clean.xlsx")

dane.head()

,Extraction_Time,Extraction_Weekday,Flight_weekday,Flight_date,Departure_time,Arrival_time,Flight_time,Departure_airport_code,arr_city,dep_city,...,Cabin_bag,Checked_bag,Days_to_departure,layover_airport,layover_duration,Airline1,low_cost1,Airline2,low_cost2,ujemne
0,02/11/2024 19:16,Saturday,Monday,2024-11-04,16.58,17.92,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
1,02/11/2024 19:16,Saturday,Monday,2024-11-04,18.33,22.67,4.33,WAW,Berlin,Warszawa,...,0,0,2,CPH,1.583333,Wizz Air,low-cost,easyJet,low-cost,-2
2,02/11/2024 19:16,Saturday,Monday,2024-11-04,19.67,21.00,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
3,02/11/2024 19:16,Saturday,Monday,2024-11-04,6.08,11.92,5.83,WAW,Berlin,Warszawa,...,0,0,2,FCO,1.083333,Wizz Air,low-cost,easyJet,low-cost,-2
4,02/11/2024 19:16,Saturday,Monday,2024-11-04,17.00,22.67,5.67,WAW,Berlin,Warszawa,...,1,0,2,MUC,2.833333,Lufthansa,legacy,Lufthansa,legacy,-2


Zamiana nazw kolumn.

In [28]:
dane = dane.rename(columns={"#Layovers": "Num_Layovers", "Price [PLN]": "Price"})
dane.columns

Index(['Extraction_Time', 'Extraction_Weekday', 'Flight_weekday',
       'Flight_date', 'Departure_time', 'Arrival_time', 'Flight_time',
       'Departure_airport_code', 'arr_city', 'dep_city',
       'Departure_airport_name', 'Destination_airport_code',
       'Destination_airport_name', 'Price', 'Num_Layovers', 'Ticket_class',
       'Cabin_bag', 'Checked_bag', 'Days_to_departure', 'layover_airport',
       'layover_duration', 'Airline1', 'low_cost1', 'Airline2', 'low_cost2',
       'ujemne'],
      dtype='object')

In [29]:
# przekształcenie dat
dane["Flight_date"] = pd.to_datetime(dane["Flight_date"])
dane["Extraction_Time"] = pd.to_datetime(dane["Extraction_Time"].str.split(" ").apply(lambda x: x[0]), dayfirst=True)


dane.head()

,Extraction_Time,Extraction_Weekday,Flight_weekday,Flight_date,Departure_time,Arrival_time,Flight_time,Departure_airport_code,arr_city,dep_city,...,Cabin_bag,Checked_bag,Days_to_departure,layover_airport,layover_duration,Airline1,low_cost1,Airline2,low_cost2,ujemne
0,2024-11-02,Saturday,Monday,2024-11-04,16.58,17.92,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
1,2024-11-02,Saturday,Monday,2024-11-04,18.33,22.67,4.33,WAW,Berlin,Warszawa,...,0,0,2,CPH,1.583333,Wizz Air,low-cost,easyJet,low-cost,-2
2,2024-11-02,Saturday,Monday,2024-11-04,19.67,21.00,1.33,WAW,Berlin,Warszawa,...,1,0,2,-,0.000000,LOT,legacy,-,-,-2
3,2024-11-02,Saturday,Monday,2024-11-04,6.08,11.92,5.83,WAW,Berlin,Warszawa,...,0,0,2,FCO,1.083333,Wizz Air,low-cost,easyJet,low-cost,-2
4,2024-11-02,Saturday,Monday,2024-11-04,17.00,22.67,5.67,WAW,Berlin,Warszawa,...,1,0,2,MUC,2.833333,Lufthansa,legacy,Lufthansa,legacy,-2


In [30]:
# usunięcie zbędnych (czy na pewno?) kolumn
dane.drop(columns = ["Extraction_Time", "Flight_date", "arr_city", "dep_city", 
                     "Departure_airport_name", "Destination_airport_name", 
                     "layover_airport", "ujemne", "low_cost1", "low_cost2"], inplace=True)

In [31]:
from  sklearn.preprocessing import OneHotEncoder 
#ohe = OneHotEncoder().fit(dane[["Extraction_Weekday"]])

Funkcja wykonująca onehotencoding zmiennych nominalnych - kategorycznych. Zwraca dataframe zer i jedynek. 

In [32]:
def onehotencode(data: pd.Series, column_to_drop):
    ohe = OneHotEncoder().fit(data)
    colnames = ["Is_" + category_name for category_name in ohe.categories_]
    
    return pd.DataFrame(data = ohe.transform(data).toarray(), columns=colnames)



In [33]:

onehot_ticket_class = pd.get_dummies(dane["Ticket_class"], prefix="Is", drop_first=True)
onehot_departure = pd.get_dummies(dane[ "Departure_airport_code"], prefix= "Is_Departure", drop_first=False)
onehot_arrival = pd.get_dummies(dane, columns=["Destination_airport_code"], prefix= "Is_Arrival", drop_first=False)

onehot_flight_weekday = pd.get_dummies(dane["Flight_weekday"], prefix = "Is_Flight", drop_first=True)
onehot_extraction_weekday = pd.get_dummies(dane["Extraction_Weekday"], prefix = "Is_Extraction", drop_first=True)

onehot_airline1 = pd.get_dummies(dane["Airline1"], prefix="Is", drop_first=False)
onehot_airline1.columns = [x + "1" for x in onehot_airline1.columns]

onehot_airline2 = pd.get_dummies(dane["Airline2"], prefix="Is", drop_first=False)
onehot_airline2.columns = [x + "2" for x in onehot_airline2.columns]


dane = pd.concat([dane, onehot_ticket_class, onehot_flight_weekday, onehot_extraction_weekday, onehot_airline1, onehot_airline2], axis=1)

dane.columns

Index(['Extraction_Weekday', 'Flight_weekday', 'Departure_time',
       'Arrival_time', 'Flight_time', 'Departure_airport_code',
       'Destination_airport_code', 'Price', 'Num_Layovers', 'Ticket_class',
       'Cabin_bag', 'Checked_bag', 'Days_to_departure', 'layover_duration',
       'Airline1', 'Airline2', 'Is_Ekonomiczna', 'Is_Flight_Monday',
       'Is_Flight_Saturday', 'Is_Flight_Sunday', 'Is_Flight_Thursday',
       'Is_Flight_Tuesday', 'Is_Flight_Wednesday', 'Is_Extraction_Monday',
       'Is_Extraction_Saturday', 'Is_Extraction_Sunday',
       'Is_Extraction_Thursday', 'Is_Extraction_Tuesday',
       'Is_Extraction_Wednesday', 'Is_Air France1', 'Is_British Airways1',
       'Is_KLM1', 'Is_LOT1', 'Is_Lufthansa1', 'Is_Ryanair1', 'Is_Wizz Air1',
       'Is_easyJet1', 'Is_-2', 'Is_Air France2', 'Is_British Airways2',
       'Is_KLM2', 'Is_LOT2', 'Is_Lufthansa2', 'Is_Ryanair2', 'Is_Wizz Air2',
       'Is_easyJet2'],
      dtype='object')

In [34]:
print(dane.columns)

Index(['Extraction_Weekday', 'Flight_weekday', 'Departure_time',
       'Arrival_time', 'Flight_time', 'Departure_airport_code',
       'Destination_airport_code', 'Price', 'Num_Layovers', 'Ticket_class',
       'Cabin_bag', 'Checked_bag', 'Days_to_departure', 'layover_duration',
       'Airline1', 'Airline2', 'Is_Ekonomiczna', 'Is_Flight_Monday',
       'Is_Flight_Saturday', 'Is_Flight_Sunday', 'Is_Flight_Thursday',
       'Is_Flight_Tuesday', 'Is_Flight_Wednesday', 'Is_Extraction_Monday',
       'Is_Extraction_Saturday', 'Is_Extraction_Sunday',
       'Is_Extraction_Thursday', 'Is_Extraction_Tuesday',
       'Is_Extraction_Wednesday', 'Is_Air France1', 'Is_British Airways1',
       'Is_KLM1', 'Is_LOT1', 'Is_Lufthansa1', 'Is_Ryanair1', 'Is_Wizz Air1',
       'Is_easyJet1', 'Is_-2', 'Is_Air France2', 'Is_British Airways2',
       'Is_KLM2', 'Is_LOT2', 'Is_Lufthansa2', 'Is_Ryanair2', 'Is_Wizz Air2',
       'Is_easyJet2'],
      dtype='object')


Usuwamy kolumny zbędne po onehotencodingu.

In [35]:
cols = ["Ticket_class", "Departure_airport_code", "Destination_airport_code", "Flight_weekday", "Extraction_Weekday", "Airline1", "Airline2", "Is_-2"]

for col in cols:
    if col not in dane.columns:
        print(col)
        break

In [36]:
dane.drop(columns = ["Ticket_class", "Departure_airport_code",
                      "Destination_airport_code", "Flight_weekday", 
                      "Extraction_Weekday", "Airline1", "Airline2", "Is_-2"], inplace=True)

In [37]:
dane.head()

,Departure_time,Arrival_time,Flight_time,Price,Num_Layovers,Cabin_bag,Checked_bag,Days_to_departure,layover_duration,Is_Ekonomiczna,...,Is_Wizz Air1,Is_easyJet1,Is_Air France2,Is_British Airways2,Is_KLM2,Is_LOT2,Is_Lufthansa2,Is_Ryanair2,Is_Wizz Air2,Is_easyJet2
0,16.58,17.92,1.33,1643,0,1,0,2,0.000000,True,...,False,False,False,False,False,False,False,False,False,False
1,18.33,22.67,4.33,1087,1,0,0,2,1.583333,True,...,True,False,False,False,False,False,False,False,False,True
2,19.67,21.00,1.33,1749,0,1,0,2,0.000000,True,...,False,False,False,False,False,False,False,False,False,False
3,6.08,11.92,5.83,1776,1,0,0,2,1.083333,True,...,True,False,False,False,False,False,False,False,False,True
4,17.00,22.67,5.67,1878,1,1,0,2,2.833333,True,...,False,False,False,False,False,False,True,False,False,False


Konwersja całego dataframe na typ numeric i podział na zbiory treningowe i testowe oraz wektor celu i macierz cech.

In [ ]:
from sklearn.model_selection import train_test_split
dane = dane.apply(pd.to_numeric)
dane.to_pickle("dane_onehot.pkl")

In [1]:
dane = pd.read_excel("loty_clean.xlsx")

NameError: name 'pd' is not defined